In [1]:
import time
import torch
import torchvision
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from ultralytics import YOLO
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights

In [6]:
# 1. Define the Image Sources
image_urls = [

    "https://ultralytics.com/images/zidane.jpg",
    "https://ultralytics.com/images/bus.jpg",
    

    "https://commons.wikimedia.org/wiki/Special:FilePath/Red_Kitten_01.jpg",
    "https://commons.wikimedia.org/wiki/Special:FilePath/Axis_axis_crossing_the_road.JPG",
    

    "https://commons.wikimedia.org/wiki/Special:FilePath/Wooden_table_and_chairs_on_a_balcony_over_the_Mekong_at_sunrise_in_Don_Det_Si_Phan_Don_Laos.jpg",
    "https://commons.wikimedia.org/wiki/Special:FilePath/Lava_lamp_on_windowsill.jpg",
    

    "https://commons.wikimedia.org/wiki/Special:FilePath/Sukhoi_SuperJet_100_(5114478300).jpg",
 
    "https://commons.wikimedia.org/wiki/Special:FilePath/Water_reflection_of_mountains_and_hut_in_a_paddy_field_with_blue_sky_in_Vang_Vieng,_Laos.jpg",

    "https://commons.wikimedia.org/wiki/Special:FilePath/Bowl_of_fruit.jpg",

    "https://commons.wikimedia.org/wiki/Special:FilePath/Street_Traffic_In_Barcelona_(166082009).jpeg"
]

In [7]:
# 2. Initialize Models
# Model A: YOLOv8 (nano version for speed)
yolo_model = YOLO('yolov8n.pt') 

# Model B: Faster R-CNN
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
frcnn_model = fasterrcnn_resnet50_fpn(weights=weights)
frcnn_model.eval()
frcnn_transform = weights.transforms()

# Store results here
results_data = []

# Define headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [8]:
#Processing

for i, url in enumerate(image_urls):
    try:
        # Load Image
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status() # Check for download errors
        img_raw = Image.open(BytesIO(response.content)).convert("RGB")
        img_name = f"Image_{i+1}"
        
        # RUN MODEL 1: YOLOv8
        start_time = time.time()
        yolo_results = yolo_model(img_raw, verbose=False)
        yolo_time = time.time() - start_time
        
        #Extract YOLO Data
        y_count = len(yolo_results[0].boxes)
        y_conf = yolo_results[0].boxes.conf.mean().item() if y_count > 0 else 0
        
        results_data.append({
            "Image": img_name,
            "Model": "YOLOv8",
            "Time (sec)": round(yolo_time, 4),
            "Objects Detected": y_count,
            "Avg Probability": round(y_conf, 4)
        })

        # RUN MODEL 2: Faster R-CNN
        img_tensor = frcnn_transform(img_raw).unsqueeze(0)
        
        start_time = time.time()
        with torch.no_grad():
            frcnn_results = frcnn_model(img_tensor)
        frcnn_time = time.time() - start_time
        
        # Extract RCNN Data (Threshold > 0.5)
        pred_scores = frcnn_results[0]['scores']
        high_conf_indices = [idx for idx, score in enumerate(pred_scores) if score > 0.5]
        
        r_count = len(high_conf_indices)
        if r_count > 0:
            r_conf = pred_scores[high_conf_indices].mean().item()
        else:
            r_conf = 0
            
        results_data.append({
            "Image": img_name,
            "Model": "Faster R-CNN",
            "Time (sec)": round(frcnn_time, 4),
            "Objects Detected": r_count,
            "Avg Probability": round(r_conf, 4)
        })
        
        print(f"Processed {img_name}...")

    except Exception as e:
        print(f"Error processing {url}: {e}")

Processed Image_1...
Processed Image_2...
Processed Image_3...
Processed Image_4...
Processed Image_5...
Processed Image_6...
Processed Image_7...
Processed Image_8...
Processed Image_9...
Processed Image_10...


In [10]:
# --- OUTPUT ---
df = pd.DataFrame(results_data)

# Column ordering was incorrect (e.g., 1, 10, 2, 3, etc. so this fixes ordering issues)
df['sort_key'] = df['Image'].apply(lambda x: int(x.split('_')[1]))
df = df.sort_values(by=['sort_key', 'Model'])
df = df.drop(columns=['sort_key'])

print("\nAnalysis Complete! Here is your table:")
print(df)

df.to_csv("model_comparison_results.csv", index=False)


Analysis Complete! Here is your table:
       Image         Model  Time (sec)  Objects Detected  Avg Probability
1    Image_1  Faster R-CNN      1.7023                 8           0.8444
0    Image_1        YOLOv8      0.0544                 3           0.6486
3    Image_2  Faster R-CNN      1.5916                 5           0.9880
2    Image_2        YOLOv8      0.0379                 6           0.6556
5    Image_3  Faster R-CNN      1.5362                 1           0.9912
4    Image_3        YOLOv8      0.0997                 2           0.6288
7    Image_4  Faster R-CNN      1.4906                 1           0.7379
6    Image_4        YOLOv8      0.0601                 1           0.8899
9    Image_5  Faster R-CNN      1.6281                 6           0.7576
8    Image_5        YOLOv8      0.0841                 3           0.6778
11   Image_6  Faster R-CNN      2.1361                 6           0.6877
10   Image_6        YOLOv8      0.2741                 3           0.574